<a href="https://colab.research.google.com/github/pranay8297/llm/blob/main/esm_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers evaluate datasets requests pandas sklearn
! pip install datasets
! pip install evaluate

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existin

In [1]:
import torch
from huggingface_hub import notebook_login

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

notebook_login()

# Playground - 1

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
model_checkpoint = "facebook/esm2_t30_150M_UR50D"

In [88]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model_checkpoint = "facebook/esm2_t36_3B_UR50D"
train_labels = 100; test_labels = 20
num_labels = 120  # Add 1 since 0 can be a label
hf_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t36_3B_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
hf_model.esm.encoder.layer[0].attention.self.attention_head_size

64

In [14]:
model_sd = model.state_dict()
for k, v in model_sd.items(): break

In [17]:
k, v

('esm.embeddings.word_embeddings.weight',
 tensor([[ 0.1686,  0.0169, -0.0477,  ..., -0.1536,  0.1710,  0.1385],
         [ 0.0445,  0.0205, -0.1031,  ..., -0.0916, -0.0045, -0.0030],
         [ 0.1497, -0.0624, -0.2800,  ..., -0.2927,  0.2073,  0.1564],
         ...,
         [ 0.0188,  0.1002, -0.1085,  ..., -0.0238, -0.1433,  0.0057],
         [-0.0223, -0.0260, -0.0794,  ..., -0.0039, -0.1411, -0.0453],
         [ 0.0449,  0.0241, -0.0987,  ..., -0.0847,  0.0028, -0.0046]]))

In [59]:
import math
import torch
import time

from torch import nn
from torch.nn import functional as F
from dataclasses import dataclass

@dataclass
class ESMConfig():
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    hidden_size: int = 4096 # 4 * block_size
    dropout: float = 0.0

class ESMLinearLayers(nn.Module):
    def __init__(self, nin, nout):
        super().__init__()
        self.dense = nn.Linear(nin, nout)
        self.dropout = nn.Dropout(getattr(config, "hidden_dropout_prob", 0))

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = hidden_states + input_tensor
        return hidden_states

class ESMSelfAttn(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)

        # Add a rotary embeddings here

        self.n_head = config.n_head
        assert config.n_embd % config.n_head == 0

        self.register_buffer('bias', torch.tril(
            torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x): pass

class ESMAttn(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.self = ESMSelfAttn(config)
        self.output = nn.ModuleDict(dict(dense = nn.Linear(config.n_embd, config.n_embd),
                                         dropout = nn.Dropout(p = config.dropout)))

        self.LayerNorm = nn.LayerNorm(config.n_embd)

    def forward(self, x): pass

class ESMLayers(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.attention = ESMAttn(config)
        self.intermediate = ESMLinearLayers(config.n_embd, config.hidden_size)
        self.output = ESMLinearLayers(config.hidden_size, config.n_embd)
        self.LayerNorm = nn.LayerNorm(config.n_embd)

    def forward(self, x): pass

class ESMEncoder(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.layer = nn.ModuleList([ESMLayers(config) for _ in range(config.n_layer)])
        self.emb_layer_norm_after = nn.LayerNorm(config.n_embd)

    def forward(self, x): pass

class ESM(nn.Module):

    def __init__(self, config):

        super().__init__()
        self.config = config
        self.esm = nn.ModuleDict(dict(
            embeddings = nn.ModuleDict(dict(
                word_embeddings = nn.Embedding(config.vocab_size, config.n_embd),
                position_embeddings = nn.Embedding(config.block_size, config.n_embd)
            )),
            encoder = ESMEncoder(config),
            final_layer = nn.Linear(config.n_embd, config.vocab_size)
        ))
        self.esm.final_layer.weight = self.esm.embeddings.word_embeddings.weight

    @classmethod
    def get_pretrained_config(cls, model_type = 'esm2_t33_650M_UR50D'):

        '''
        name                n_layers    n_params
        esm2_t48_15B_UR50D	48	        15B
        esm2_t36_3B_UR50D	36	        3B
        esm2_t33_650M_UR50D	33	        650M
        esm2_t30_150M_UR50D	30	        150M
        esm2_t12_35M_UR50D	12	        35M
        esm2_t6_8M_UR50D
        '''

        assert model_type in {'esm2_t36_3B_UR50D', 'esm2_t33_650M_UR50D', 'esm2_t30_150M_UR50D'}

        config_args = {
            'esm2_t36_3B_UR50D': dict(n_layer=36, n_head = 64, n_embd=2560, hidden_size=10240), # 3B params
            'esm2_t33_650M_UR50D': dict(n_layer=33, n_head = 32, n_embd=1280, hidden_size=5120), # 650M params
            'esm2_t30_150M_UR50D': dict(n_layer=30, n_head = 16, n_embd=640, hidden_size=2560), # 150M params
        }[model_type]

        config_args['vocab_size'] = 33 # always 33 for ESM Models
        config_args['block_size'] = 1026 # Always constant for ESM Models
        config = ESMConfig(**config_args)
        return config


    @classmethod
    def load_pretrained(cls, model_type = 'esm2_t33_650M_UR50D'):

        config = cls.get_pretrained_config(model_type)
        print("loading weights from pretrained gpt: %s" % model_type)

        # create a from-scratch initialized minGPT model
        model = cls(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        from transformers import AutoModelForSequenceClassification

        model_checkpoint = f"facebook/{model_type}"
        num_labels = 33
        model_hf = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        sd_keys_hf = [k for k in sd_keys_hf if 'inv_freq' not in k]
        sd_keys_hf = [k for k in sd_keys_hf if 'classifier' not in k]

        ignore_keys = ['esm.contact_head.regression.weight', 'esm.contact_head.regression.bias']
        for k in sd_keys_hf:

            if k in ignore_keys: continue

            # vanilla copy over the other parameters
            try: assert sd_hf[k].shape == sd[k].shape
            except Exception as e: print(f"Mismatch in the shape of tensor while loading weights - Key: {k}, expected shape: {sd_hf[k].shape}, actual shape: {sd[k].shape}")

            with torch.no_grad():
                sd[k].copy_(sd_hf[k])

        # Set the final layers bias as 0 so that it does not affect weight tying scheme
        with torch.no_grad():
            model.esm.final_layer.bias.zero_()

        # Freeze the model
        for param in model.parameters():
            param.requires_grad = False

        return model

    def forward(self, x): pass

In [64]:
model = ESM.load_pretrained(model_type = 'esm2_t30_150M_UR50D')

esm2_t30_150M_UR50D
loading weights from pretrained gpt: esm2_t30_150M_UR50D


Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [62]:
config = ESM.get_pretrained_config("esm2_t30_150M_UR50D")
model = ESM(config)

esm2_t30_150M_UR50D


In [72]:
with torch.no_grad():
    model.esm.final_layer.bias.zero_()
model.esm.final_layer.bias.requires_grad = False

# Data Shit

In [2]:
# Load Data
import requests

query_url ="https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession%2Csequence%2Ccc_subcellular_location&format=tsv&query=%28%28organism_id%3A9606%29%20AND%20%28reviewed%3Atrue%29%20AND%20%28length%3A%5B80%20TO%20500%5D%29%29"
uniprot_request = requests.get(query_url)

from io import BytesIO
import pandas

bio = BytesIO(uniprot_request.content)

df = pandas.read_csv(bio, compression='gzip', sep='\t')
df.head()

df = df.dropna()
cytosolic = df['Subcellular location [CC]'].str.contains("Cytosol") | df['Subcellular location [CC]'].str.contains("Cytoplasm")
membrane = df['Subcellular location [CC]'].str.contains("Membrane") | df['Subcellular location [CC]'].str.contains("Cell membrane")
cytosolic_df = df[cytosolic & ~membrane]
cytosolic_df.head()

membrane_df = df[membrane & ~cytosolic]
membrane_df.head()

cytosolic_sequences = cytosolic_df["Sequence"].tolist()
cytosolic_labels = [0 for protein in cytosolic_sequences]
membrane_sequences = membrane_df["Sequence"].tolist()
membrane_labels = [1 for protein in membrane_sequences]
sequences = cytosolic_sequences + membrane_sequences
labels = cytosolic_labels + membrane_labels

# Quick check to make sure we got it right
assert len(sequences) == len(labels)

In [3]:
model_checkpoint = 'facebook/esm2_t30_150M_UR50D'
from sklearn.model_selection import train_test_split

train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding='max_length', max_length=1026)
tokenizer(train_sequences[0])

from datasets import Dataset

train_tokenized = tokenizer(train_sequences)
test_tokenized = tokenizer(test_sequences)

padded_train_tokenized = tokenizer.pad(train_tokenized, padding = 'max_length', max_length= 1026)
padded_test_tokenized = tokenizer.pad(test_tokenized, padding = 'max_length', max_length= 1026)

padded_train_dataset = Dataset.from_dict(padded_train_tokenized)
padded_train_dataset = padded_train_dataset.add_column("labels", train_labels)

padded_test_dataset = Dataset.from_dict(padded_test_tokenized)
padded_test_dataset = padded_test_dataset.add_column("labels", test_labels)
print(len(padded_train_dataset[0]['input_ids']), len(padded_train_dataset[1]['input_ids']), len(padded_test_dataset[0]['input_ids']), len(padded_test_dataset[1]['input_ids']))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1026 1026 1026 1026


In [4]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = max(train_labels + test_labels) + 1  # Add 1 since 0 can be a label
hf_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

model_name = model_checkpoint.split("/")[-1]
batch_size = 8

args = TrainingArguments(
    f"{model_name}-finetuned-localization",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

from evaluate import load
import numpy as np

metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

from torch.utils.data import DataLoader
class MyTrainer(Trainer):

    def get_train_dataloader(self, batch_size = 2):
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = self._get_train_sampler()

        return DataLoader(
            self.train_dataset,
            batch_size=batch_size,
            sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
        )

trainer = MyTrainer(
    hf_model,
    args,
    train_dataset = padded_train_dataset,
    eval_dataset = padded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

train_dl = trainer.get_train_dataloader(2)
batch = next(iter(train_dl))

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Playground - 2

In [5]:
import math
import torch

from torch import nn
from torch.nn import functional as F
from dataclasses import dataclass
from einops import rearrange, repeat


def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(x, cos, sin):
    cos = cos[:, :, : x.shape[-2], :]
    sin = sin[:, :, : x.shape[-2], :]

    return (x * cos) + (rotate_half(x) * sin)

class RotaryEmbedding(torch.nn.Module):
    """
    Rotary position embeddings based on those in
    [RoFormer](https://huggingface.co/docs/transformers/model_doc/roformer). Query and keys are transformed by rotation
    matrices which depend on their relative positions.
    """

    def __init__(self, dim: int):
        super().__init__()
        # Generate and save the inverse frequency buffer (non trainable)
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2, dtype=torch.int64).float() / dim))
        inv_freq = inv_freq
        self.register_buffer("inv_freq", inv_freq)

        self._seq_len_cached = None
        self._cos_cached = None
        self._sin_cached = None

    def _update_cos_sin_tables(self, x, seq_dimension=2):
        seq_len = x.shape[seq_dimension]

        # Reset the tables if the sequence length has changed,
        # or if we're on a new device (possibly due to tracing for instance)
        if seq_len != self._seq_len_cached or self._cos_cached.device != x.device:
            self._seq_len_cached = seq_len
            t = torch.arange(x.shape[seq_dimension], device=x.device).type_as(self.inv_freq)
            freqs = torch.outer(t, self.inv_freq)
            emb = torch.cat((freqs, freqs), dim=-1).to(x.device)

            self._cos_cached = emb.cos()[None, None, :, :]
            self._sin_cached = emb.sin()[None, None, :, :]

        return self._cos_cached, self._sin_cached

    def forward(self, q: torch.Tensor, k: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        self._cos_cached, self._sin_cached = self._update_cos_sin_tables(k, seq_dimension=-2)

        return (
            apply_rotary_pos_emb(q, self._cos_cached, self._sin_cached),
            apply_rotary_pos_emb(k, self._cos_cached, self._sin_cached)
        )

@dataclass
class ESMConfig():
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    hidden_size: int = 4096 # 4 * block_size
    dropout: float = 0.0

class ESMIntermediateLayer(nn.Module):
    def __init__(self, nin, nout, dropout = 0.0):
        super().__init__()
        self.dense = nn.Linear(nin, nout)
        self.act = nn.GELU(approximate = 'tanh')

    def forward(self, x):
        return self.act(self.dense(x))

class ESMOutLayer(nn.Module):
    def __init__(self, nin, nout, dropout = 0.0):
        super().__init__()
        self.dense = nn.Linear(nin, nout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, attn_scores):
        x = self.dense(x)
        x = self.dropout(x)
        x = x + attn_scores
        return x

class ESMSelfAttn(nn.Module): # Verified

    def __init__(self, config):
        super().__init__()

        assert config.n_embd % config.n_head == 0

        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)

        self.n_head = config.n_head
        attention_head_size = config.n_embd//config.n_head
        # Add a rotary embeddings here
        self.rotary_embeddings = RotaryEmbedding(dim = attention_head_size)

    def forward(self, x, attention_mask):

        # x -> (b, s, e) -> (b s, h, e/h)
        k, q, v = self.key(x), self.query(x), self.value(x)

        k = rearrange(k, 'b s (h e) -> b h s e', h = self.n_head)
        q = rearrange(q, 'b s (h e) -> b h s e', h = self.n_head)
        v = rearrange(v, 'b s (h e) -> b h s e', h = self.n_head)

        # Add rotary embeddings here for k and q tensors
        q, k = self.rotary_embeddings(q, k)

        # Attention claculation - # TODO: make is_casual true in case of finetuning - Very important
        y = F.scaled_dot_product_attention(q, k, v, attn_mask = attention_mask, is_causal=False) # flash attention
        y = rearrange(y, 'b h s e -> b s (h e)', h = self.n_head)
        return y

class ESMAttn(nn.Module): # Verified

    def __init__(self, config):
        super().__init__() # No activation function at this level
        self.self = ESMSelfAttn(config)
        self.output = ESMOutLayer(config.n_embd, config.n_embd, dropout = getattr(config, 'dropout', 0.))
        self.LayerNorm = nn.LayerNorm(config.n_embd)

    def forward(self, x, attention_mask):
        inter_x = self.LayerNorm(x)
        attn = self.self(inter_x, attention_mask)
        out = self.output(attn, x)
        return out

class ESMLayers(nn.Module): # Both Init and Forward Verified - Done and Dusted

    def __init__(self, config):
        super().__init__()
        self.attention = ESMAttn(config)
        self.intermediate = ESMIntermediateLayer(config.n_embd, config.hidden_size) #
        self.output = ESMOutLayer(config.hidden_size, config.n_embd) #
        self.LayerNorm = nn.LayerNorm(config.n_embd)

    def forward(self, x, attention_mask):
        attention_op = self.attention(x, attention_mask)
        attention_op_ln = self.LayerNorm(attention_op)
        inter = self.intermediate(attention_op_ln)
        out = self.output(inter, attention_op)
        return out

class ESMEncoder(nn.Module):

    def __init__(self, config):
        super().__init__()

        # No activation functions here as well

        self.layer = nn.ModuleList([ESMLayers(config) for _ in range(config.n_layer)])
        self.emb_layer_norm_after = nn.LayerNorm(config.n_embd)

    def forward(self, x, attention_mask = None):

        for layer in self.layer:
            x = layer(x, attention_mask)

        return self.emb_layer_norm_after(x)

class ESM(nn.Module):

    def __init__(self, config):

        super().__init__()
        self.config = config
        self.esm = nn.ModuleDict(dict(
            embeddings = nn.ModuleDict(dict( # No activation needed - for both the embeddings - done - forward done
                word_embeddings = nn.Embedding(config.vocab_size, config.n_embd),
                position_embeddings = nn.Embedding(config.block_size, config.n_embd)
            )),
            encoder = ESMEncoder(config), # Done, forward - here
            final_layer = nn.Linear(config.n_embd, config.vocab_size)
        ))
        self.esm.final_layer.weight = self.esm.embeddings.word_embeddings.weight

    @classmethod
    def get_pretrained_config(cls, model_type = 'esm2_t33_650M_UR50D'):

        '''
        name                n_layers    n_params
        esm2_t48_15B_UR50D	48	        15B
        esm2_t36_3B_UR50D	36	        3B
        esm2_t33_650M_UR50D	33	        650M
        esm2_t30_150M_UR50D	30	        150M
        esm2_t12_35M_UR50D	12	        35M
        esm2_t6_8M_UR50D
        '''

        assert model_type in {'esm2_t36_3B_UR50D', 'esm2_t33_650M_UR50D', 'esm2_t30_150M_UR50D'}

        config_args = {
            'esm2_t36_3B_UR50D': dict(n_layer=36, n_head = 40, n_embd=2560, hidden_size=10240), # 3B params
            'esm2_t33_650M_UR50D': dict(n_layer=33, n_head = 20, n_embd=1280, hidden_size=5120), # 650M params
            'esm2_t30_150M_UR50D': dict(n_layer=30, n_head = 20, n_embd=640, hidden_size=2560), # 150M params
        }[model_type]

        config_args['vocab_size'] = 33 # always 33 for ESM Models
        config_args['block_size'] = 1026 # Always constant for ESM Models
        config = ESMConfig(**config_args)
        return config


    @classmethod
    def load_pretrained(cls, model_type = 'esm2_t33_650M_UR50D'):

        config = cls.get_pretrained_config(model_type)
        print("loading weights from pretrained gpt: %s" % model_type)

        # create a from-scratch initialized minGPT model
        model = cls(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        from transformers import AutoModelForSequenceClassification

        model_checkpoint = f"facebook/{model_type}"
        num_labels = 33
        model_hf = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        sd_keys_hf = [k for k in sd_keys_hf if 'inv_freq' not in k]
        sd_keys_hf = [k for k in sd_keys_hf if 'classifier' not in k]

        ignore_keys = ['esm.contact_head.regression.weight', 'esm.contact_head.regression.bias']
        for k in sd_keys_hf:

            if k in ignore_keys: continue

            # vanilla copy over the other parameters
            try: assert sd_hf[k].shape == sd[k].shape
            except Exception as e: print(f"Mismatch in the shape of tensor while loading weights - Key: {k}, expected shape: {sd_hf[k].shape}, actual shape: {sd[k].shape}")

            with torch.no_grad():
                sd[k].copy_(sd_hf[k])

        # Set the final layers bias as 0 so that it does not affect weight tying scheme
        with torch.no_grad():
            model.esm.final_layer.bias.zero_()

        # Freeze the model
        for param in model.parameters():
            param.requires_grad = False

        return model

    def get_embs(self, x, attention_mask = None):
        token_embs = self.esm.embeddings.word_embeddings(x)
        # Not required as we are use rotary embeddings - Hence we do not require absolute position embeddings
        # position_embs = self.esm.embeddings.position_embeddings(torch.arange(0, x.shape[1], 1, dtype = torch.long))

        if attention_mask is not None:
            token_embs = (token_embs * attention_mask.unsqueeze(-1)).to(token_embs.dtype)

        return token_embs

    def get_extended_attn_mask(self, attention_mask, input_shape):

        if attention_mask == None: return None
        b, s = attention_mask.shape
        # Make the attention mask braodcastable for [batch_size, n_heads, seq_len, seq_len]
        attention_mask = attention_mask[:, None, None, :]

        # Now make sure that it has negetive infinity for all the padded tokens and
        # 0 for all attention tokens as we add this mask to attention scores
        attn_mask = attention_mask.to(torch.float32)
        attn_mask = (1 - attn_mask) * (torch.finfo(torch.float32).min)
        attn_mask = attn_mask.expand(b, 1, s, s)
        return attn_mask

    def forward(self, x, y = None, attention_mask = None, output_encoder = True):

        # Calculate Embeddings
        x = self.get_embs(x, attention_mask) # Verified

        # compute attention_mask for attention scores
        attention_mask = self.get_extended_attn_mask(attention_mask, x.shape)

        #Do the forward pass
        x = self.esm.encoder(x, attention_mask = attention_mask)
        if not output_encoder:
            return self.esm.final_layer(x)
        else:
            return self.esm.final_layer(x), x

model = ESM.load_pretrained("esm2_t30_150M_UR50D") # load the pretrained frozen model
print('Models created')
print(model)

loading weights from pretrained gpt: esm2_t30_150M_UR50D


Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Models created
ESM(
  (esm): ModuleDict(
    (embeddings): ModuleDict(
      (word_embeddings): Embedding(33, 640)
      (position_embeddings): Embedding(1026, 640)
    )
    (encoder): ESMEncoder(
      (layer): ModuleList(
        (0-29): 30 x ESMLayers(
          (attention): ESMAttn(
            (self): ESMSelfAttn(
              (query): Linear(in_features=640, out_features=640, bias=True)
              (key): Linear(in_features=640, out_features=640, bias=True)
              (value): Linear(in_features=640, out_features=640, bias=True)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): ESMOutLayer(
              (dense): Linear(in_features=640, out_features=640, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((640,), eps=1e-05, elementwise_affine=True)
          )
          (intermediate): ESMIntermediateLayer(
            (dense): Linear(in_features=640, out_features=2560, bi

# Verification of the model equivalence - Done

`The encoder outputs are verified and margin of error is less than 1%`

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
hf_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t30_150M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [105]:
# hf_model.esm.encoder.layer = nn.ModuleList([hf_model.esm.encoder.layer[0]])
# model.esm.encoder.layer = nn.ModuleList([model.esm.encoder.layer[0]])

In [8]:
hf_model = hf_model.to(device)
hf_model = hf_model.eval()
hf_model.esm.embeddings.token_dropout = False
batch['input_ids'], batch['attention_mask'] = batch['input_ids'].to(device), batch['attention_mask'].to(device)

In [111]:
class Hook:
    def __init__(self, module):

        self.output = None
        self.input = None

        def hook_fn(module, input, output, hook_obj = self):
            hook_obj.output = output
            hook_obj.input = input

        self.h = module.register_forward_hook(hook_fn)


hook_obj = Hook(hf_model.esm.encoder)

In [108]:
out = hf_model(batch['input_ids'], batch['attention_mask'])

In [13]:
batch['attention_mask'][:, None, None, :].shape

torch.Size([2, 1, 1, 1026])

In [9]:
embs = hf_model.esm.embeddings(batch['input_ids'], attention_mask = batch['attention_mask'])
extended_attn_mask = hf_model.esm.get_extended_attention_mask(batch["attention_mask"], batch['input_ids'].shape).to(device)

embs.shape, extended_attn_mask.shape

(torch.Size([2, 1026, 640]), torch.Size([2, 1, 1, 1026]))

In [10]:
hf_encoder_output = hf_model.esm.encoder(embs, extended_attn_mask)

In [11]:
hf_encoder_output.last_hidden_state.shape

torch.Size([2, 1026, 640])

In [14]:
# out = model(batch['input_ids'].cpu(), batch['attention_mask'].cpu())
x = model.get_embs(batch['input_ids'].cpu(), batch['attention_mask'].cpu()) # Verified
# compute attention_mask for attention scores
attention_mask = model.get_extended_attn_mask(batch['attention_mask'].cpu(), x.shape)
x.shape, attention_mask.shape

(torch.Size([2, 1026, 640]), torch.Size([2, 1, 1026, 1026]))

In [15]:
encoder_output = model.esm.encoder(x, attention_mask)

In [16]:
encoder_output.shape

torch.Size([2, 1026, 640])

In [17]:
diff = hf_encoder_output.last_hidden_state.cpu().detach().abs() - encoder_output.abs()

In [18]:
diff.mean(), diff.std()

(tensor(-8.1190e-05), tensor(0.0045))

In [20]:
diff.mean()/hf_encoder_output.last_hidden_state.cpu().detach().abs().mean()

tensor(-0.0005)

In [21]:
model_output = model(batch['input_ids'].cpu(), batch['attention_mask'].cpu())

In [22]:
type(model_output)

tuple

In [23]:
model_output[0].shape

torch.Size([2, 1026, 33])

In [24]:
model_output[0][0,:10, :]

tensor([[-4.8848e-02, -1.3215e+00, -1.8367e+00, -1.3063e+00,  7.3408e-01,
         -2.7227e-01,  1.6907e-01, -2.2019e-02, -2.5513e-01, -6.4382e-01,
          2.0296e-01, -4.5170e-02, -2.4302e+00, -1.4317e-01,  6.4844e-01,
          7.6697e-03, -1.9361e-01,  1.6448e-01, -5.0863e-01, -1.8080e-01,
          9.8537e-01, -2.0467e-01, -2.7810e-01, -2.0719e-01,  8.8968e-01,
          1.9978e-01, -1.2735e-02, -2.1549e-01, -4.3526e-01, -7.2080e-01,
         -8.1883e-02, -1.1430e-01, -1.2763e+00],
        [-1.5846e+00, -6.9800e-01, -1.8844e+00, -6.5742e-01,  2.6395e-01,
          4.4939e-01, -2.7843e-02, -5.1529e-02,  9.7963e-01, -6.6084e-01,
         -1.0287e+00,  2.1330e-01, -1.4997e+00, -2.9639e-01, -5.5355e-01,
         -6.7088e-01, -1.0794e+00,  1.4595e+00,  6.1735e-01,  3.6264e-01,
          1.3268e+00, -1.0690e+00,  4.9841e-01,  5.2764e-02,  1.2637e+00,
          3.4913e-02, -2.6115e-01, -2.1444e-01,  2.9574e-01, -8.1107e-01,
         -1.5118e-01, -1.7452e-01, -6.0457e-01],
        [-1.19